In [ ]:
# # install octave
# !sudo apt-get -qq update
# !sudo apt-get -qq install octave octave-signal liboctave-dev

# # install oct2py that compatible with colab
# import os

# from pkg_resources import get_distribution

# os.system(
#     f"pip install -qq"
#     f" ipykernel=={get_distribution('ipykernel').version}"
#     f" ipython=={get_distribution('ipython').version}"
#     f" tornado=={get_distribution('tornado').version}"
#     f" oct2py"
# )

# # install packages
# !pip install -qq matpower matpowercaseframes

# Impedance Refactor for Parallel Lines

## Objective
To model a single impedance $ Z $ as $ N $ parallel lines in a power system data file.

## Formula

### Impedance

For $ N $ parallel lines with equal impedance:

$$
\frac{1}{Z_{\text{total}}} = \frac{1}{Z_1} + \frac{1}{Z_2} + \dots + \frac{1}{Z_n} = \frac{N}{Z_1} \quad \Rightarrow \quad Z_{\text{total}} = \frac{Z_1}{N}
$$

To refactor the impedance for each line, multiply the original impedance $ Z $ by $ N $:

$$
Z_n = Z \times N
$$

Where $ Z = R + jX $ (resistance and reactance).

### Susceptance

For $ N $ parallel lines with equal susceptance:

$$
B_{\text{total}} = B_1 + B_2 + \dots + B_n = N \times B_1
$$

To refactor the susceptance for each line, divide the original susceptance $ B $ by $ N $:

$$
B_{\text{new}} = \frac{B}{N}
$$

### Rate

For $ N $ parallel lines with equal rate $ C $:

$$
C_{\text{total}} = C_{1} + C_{2} + \dots + C_{n} = N \times C_1
$$

To refactor the rate for each line, divide the original rate $ C $ by $ N $:

$$
C_n = \frac{C}{N}
$$

Where $ C $ long term, short term and emergency rating in MVA.

## Example
Original impedance $ Z = 0.01 + j0.02 $ with line capacity $ C = 75 $ and total line charging susceptance $ B = 0.15$ , splitted into three parallel lines $ N = 3 $:

$$
Z_{\text{new}} = Z \times N = (0.01 + j0.02) \times 3 = 0.03 + j0.06
$$

$$
B_{\text{new}} = B / N = 0.15 / 3 = 0.05
$$

$$
C_{\text{new}} = C / N = 75 / 3 = 25
$$

We then have N row in branch data, with each using $ Z_{\text{new}} $, $ B_{\text{new}} $, and $ C_{\text{new}} $.

## Understanding Parallel Line

$$
Y_{total} = Y_1 + Y_2 + \dots + Y_n
$$

In [ ]:
import matpowercaseframes
import oct2py

import matpower

print(f"matpowercaseframes version: {matpowercaseframes.__version__}")
print(f"oct2py version: {oct2py.__version__}")

print(f"matpower version: {matpower.__version__}")

In [ ]:
import os

import pandas as pd
from matpowercaseframes import CaseFrames
from matpowercaseframes.constants import COLUMNS

from matpower import path_matpower, start_instance

## Inspect data

In [ ]:
m = start_instance()

In [ ]:
path = os.path.join(path_matpower, "data/case9.m")
cf = CaseFrames(path, load_case_engine=m)
cf.infer_numpy()
cf.branch

## Run base

In [ ]:
mpc_org = cf.to_mpc()
sol = m.runpf(mpc_org, verbose=True)

In [ ]:
pd.DataFrame(sol["branch"], columns=COLUMNS["branch"][:17])

## Edit branch to became N parallel lines

In [ ]:
N = 2
cf.branch = pd.concat([cf.branch] * N, ignore_index=True)
cf.branch.loc[:, ["BR_R", "BR_X"]] = cf.branch.loc[:, ["BR_R", "BR_X"]] * N
cf.branch.loc[:, ["BR_B", "RATE_A", "RATE_B", "RATE_C"]] = (
    cf.branch.loc[:, ["BR_B", "RATE_A", "RATE_B", "RATE_C"]] / N
)
cf.branch

In [ ]:
mpc_new = cf.to_mpc()
sol = m.runpf(mpc_new, verbose=True)

In [ ]:
pd.DataFrame(sol["branch"], columns=COLUMNS["branch"][:17])

## TODO: Show the new branch resiliency when there is branch contingency N - 1